In [1]:
import sys
sys.path.append('/Users/mike/Dropbox/science/_projects/')
sys.path.append('/Users/mike/github/QDMlab')
sys.path.append(r"C:\Users\micha\github\QDMlab")
sys.path.append(r"C:\Users\micha\Dropbox\science\_projects")

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import numpy as np
from pandas import ExcelWriter
import admin_tools
from admin_tools import arguments, docstring

In [ ]:
code = admin_tools.preload_code(reload=True)

In [ ]:
def call(file):
    return ' '+file[:-2]+'('

# number of functions

In [ ]:
for package, files in code.items():
    n = len(files.keys())
    print(f'{package:50} {n:3} functions')

In [ ]:
[i for i,v in enumerate(docs['fit_resonance']) if 'parameters' in v.lower()]

In [ ]:
code = admin_tools.preload_code(reload=True)
for f, doc in docs.items():
    param_idx = [i for i,v in enumerate(doc) if 'parameters' in v.lower()]
    for i in param_idx:
        try:
            print(f, i, doc[i:i+2])
        except:
            pass

In [ ]:
docs = docstring.get_docstring(reload=True)
idx = docstring.get_docstring_indices(reload=True)
# params['ODMR_to_B111']
docs['ODMR_to_B111']
idx['ODMR_to_B111']

In [ ]:
for f in sorted(params):
    for package, files in code.items():
        for file, c in files.items():
            strt_idx = [i for i, l in enumerate(c) if l.startswith('function') and f'{f in l]
            if strt_idx:
                print(f, package, file, strt_idx)

In [ ]:
def get_docstring_indices(reload = False, debug_fname=''):
    """
    Walks through all functions and gets the index for the docstrings
    """
    code = admin_tools.preload_code(reload = reload)
    params = arguments.get_in_out_params(reload = reload)
    comment_block_index = {}
    for f in params:
        for package, files in code.items():
            for file, c in files.items():
                strt_idx = [i for i, l in enumerate(c) if l.startswith('function') and f' {f}(' in l.replace('=', ' ')]
                if len(strt_idx) != 1:
                    # file does not include the function
                    continue
                else:
                    strt_idx = strt_idx[0]
                comment_idx = []
                idx = np.array([i for i, l in enumerate(c[strt_idx:]) if l.rstrip().startswith('%')])
                df = np.diff(idx, prepend=True)
                
                try:
                    until_idx = np.where(df>1)[0][0]
                except IndexError:
                    until_idx = len(idx)
                    
                comment_idx = idx[:until_idx]
                
                comment_block_index[f] = comment_idx
                
                if f == debug_fname:
                    print(f, file)
                    print(strt_idx, c[strt_idx])
                    print('comment indices: ', comment_idx)
                    print()
                    print('extracted comments')
                    print('------------------')
                    for l in np.array(c)[comment_idx]:
                        print(l, end='')
                    
    return comment_block_index

In [ ]:
ds = get_docstring_indices(debug_fname='B111')

In [ ]:
ds = docstring.get_docstring_indices(debug_fname='B111')

In [ ]:
docs = docstring.get_docstring(reload=True)
params = admin_tools.get_in_out_params(reload=True)
param_docs = {f:{} for f in docs.keys()}
n = 0

for f, doc in docs.items():
    if f == 'estimate_coercivity':
        continue
        
    param_idx = [i for i,v in enumerate(doc) if 'parameters' in v.lower()]
    dash_idx = [i for i,v in enumerate(doc) if '---' in v.lower()]
    
    idx = [i for i in param_idx if i+1 in dash_idx]
    
    if f == 'ODMR_to_B111':
        print(param_idx, dash_idx)
    if len(idx) != 1:
        print('='*100)
        print(f'{f:50}!!! NO PARAMETERS IN DOCSTRING')
        print('>>> ', params[f]['funcPath'])
        try:
            # print()
            docstring.make_params_doc(params[f])
        except:
            print('>>>> cant make docstring')
        n += 1
        print()
        
    else:
        doc = doc[param_idx[0]+2:]
        colon_idx = [i for i,v in enumerate(doc) if ': ' in v if not 'Note:' in v if not 'See Also:' in v if not ' if ' in v]
        colon_idx.append(len(doc))
        
        for i, idx in enumerate(colon_idx[:-1]):
            d = doc[idx: colon_idx[i+1]]
            p = ''.join(d).split(':')[0][1:].lstrip()
            param_docs[f][p] = ''.join(d)
print(f'{n}/{len(docs.keys())} functions without parameter descriptions')

# check where functions are called 

In [ ]:
for package, files in code.items():
    print(package)
    print('='*50)    
    for file in files.keys():
        print('>>>>>', file)
        n = 0
        for _, files in code.items():
            for f, c in files.items():
                if f == file:
                    continue
                if f.startswith('test_'):
                    continue
                if call(file) in ''.join(c):
                    n += 1
                    print(f'     - {file} called in {f}')
                    idx = [i for i,l in enumerate(c) if call(file) in l]
                    for i in idx:
                        print(f'            {c[i].rstrip().lstrip()}')
        if n == 0:
            print(f'WARNING: {file} is never called')
        print('-'*50)
    print('='*50)    

# check where functions are called 

In [ ]:
for package, files in code.items():
    print(package)
    print('='*50)    
    for file in files.keys():
        print('>>>>>', file)
        n = 0
        for _, files in code.items():
            for f, c in files.items():
                if f == file:
                    continue
                if f.startswith('test_'):
                    continue
                if call(file) in ''.join(c):
                    n += 1
                    print(f'     - {file} called in {f}')
                    idx = [i for i,l in enumerate(c) if call(file) in l]
                    for i in idx:
                        print(f'            {c[i].rstrip().lstrip()}')
        if n == 0:
            print(f'WARNING: {file} is never called')
        print('-'*50)
    print('='*50)    

In [ ]:
make_params_doc(p['fit_resonance'])

In [65]:
docs = docstring.get_docstring(reload=True)
params = admin_tools.get_in_out_params(reload=True)
param_docs = {f:{} for f in docs.keys()}
n = 0

for f, doc in docs.items():
    if f == 'estimate_coercivity':
        continue
        
    param_idx = [i for i,v in enumerate(doc) if 'parameters' in v.lower()]
    dash_idx = [i for i,v in enumerate(doc) if '---' in v.lower()]
    
    idx = [i for i in param_idx if i+1 in dash_idx]
    
    if f == 'ODMR_to_B111':
        print(param_idx, dash_idx)
    if len(idx) != 1:
        print('='*100)
        print(f'{f:50}!!! NO PARAMETERS IN DOCSTRING')
        print('>>> ', params[f]['funcPath'])
        try:
            # print()
            docstring.make_params_doc(params[f])
        except:
            print('>>>> cant make docstring')
        n += 1
        print()
        
    else:
        doc = doc[param_idx[0]+2:]
        colon_idx = [i for i,v in enumerate(doc) if ': ' in v if not 'Note:' in v if not 'See Also:' in v if not ' if ' in v]
        colon_idx.append(len(doc))
        
        for i, idx in enumerate(colon_idx[:-1]):
            d = doc[idx: colon_idx[i+1]]
            p = ''.join(d).split(':')[0][1:].lstrip()
            param_docs[f][p] = ''.join(d)
print(f'{n}/{len(docs.keys())} functions without parameter descriptions')

check_plot                                        !!! NO PARAMETERS IN DOCSTRING
>>>  C:\Users\micha\github\QDMlab\alignment_tools\get_transformed_maps.m
>>>> cant make docstring

updateRand                                        !!! NO PARAMETERS IN DOCSTRING
>>>  C:\Users\micha\github\QDMlab\analysis\globalFraction_estimator.m
>>>> cant make docstring

updatePlot                                        !!! NO PARAMETERS IN DOCSTRING
>>>  C:\Users\micha\github\QDMlab\analysis\globalFraction_estimator.m
>>>> cant make docstring

MITBxByFromBz                                     !!! NO PARAMETERS IN DOCSTRING
>>>  C:\Users\micha\github\QDMlab\map_editing\MITBxByFromBz.m
>>>> cant make docstring

QDMBzFromBu                                       !!! NO PARAMETERS IN DOCSTRING
>>>  C:\Users\micha\github\QDMlab\map_editing\QDMBzFromBu.m
>>>> cant make docstring

QuadBGsub                                         !!! NO PARAMETERS IN DOCSTRING
>>>  C:\Users\micha\github\QDMlab\map_editing\Qua

# check where functions are called 

In [10]:
for package, files in code.items():
    print(package)
    print('='*50)    
    for file in files.keys():
        print('>>>>>', file)
        n = 0
        for _, files in code.items():
            for f, c in files.items():
                if f == file:
                    continue
                if f.startswith('test_'):
                    continue
                if call(file) in ''.join(c):
                    n += 1
                    print(f'     - {file} called in {f}')
                    idx = [i for i,l in enumerate(c) if call(file) in l]
                    for i in idx:
                        print(f'            {c[i].rstrip().lstrip()}')
        if n == 0:
            print(f'WARNING: {file} is never called')
        print('-'*50)
    print('='*50)    

C:\Users\micha\github\QDMlab\alignment_tools
>>>>> align_images.m
     - align_images.m called in get_transformed_maps.m
            [nTransForms, nRefFrames] = align_images(nFolders, 'transformFile', 0, ...
--------------------------------------------------
>>>>> get_image_tform.m
     - get_image_tform.m called in align_images.m
            [tForm, refFrame] = get_image_tform(movingLed, fixedLed, 'check', true);
            [tForm, refFrame] = get_image_tform(fixedLed, movingLed, 'check', true);
     - get_image_tform.m called in get_tform_multi.m
            [tForm, refFrame] = get_image_tform(movingLed, fixedLed, 'checkPlot', checkPlot);
            [tForm, refFrame] = get_image_tform(fixedLed, movingLed, 'checkPlot', checkPlot);
     - get_image_tform.m called in save_transformed_map.m
            [transForm, refFrame] = get_image_tform(movingData.(kwargs.reference), ...
     - get_image_tform.m called in subtract_blank.m
            [transForm, refFrame] = get_image_tform(fixedDa

# check where functions are called 

In [ ]:
for package, files in code.items():
    print(package)
    print('='*50)    
    for file in files.keys():
        print('>>>>>', file)
        n = 0
        for _, files in code.items():
            for f, c in files.items():
                if f == file:
                    continue
                if f.startswith('test_'):
                    continue
                if call(file) in ''.join(c):
                    n += 1
                    print(f'     - {file} called in {f}')
                    idx = [i for i,l in enumerate(c) if call(file) in l]
                    for i in idx:
                        print(f'            {c[i].rstrip().lstrip()}')
        if n == 0:
            print(f'WARNING: {file} is never called')
        print('-'*50)
    print('='*50)    

In [85]:
make_params_doc(p['fit_resonance'])

% Parameters
% ----------
%   expData:
%   binSize:
%   nRes:
%   header:
%   header: ('none')
%   type: (2)
%   globalFraction: (0.5)
%   forceGuess: (false)
%   checkPlot: (false)
%   gaussianFit: (false)
%   gaussianFilter: (0)
%   smoothDegree: (2)
%   diamond: ('doublet')
%   slopeCorrection: (false)
%   crop: ('none')
%   fcrop: (false)
% 
% Returns
% ----------
%   fit:


In [59]:
p['fit_resonance']

{'parameters': ['expData', 'binSize', 'nRes', 'header'],
 'returns': ['fit'],
 'funcPath': 'C:\\Users\\micha\\github\\QDMlab\\fitting\\fit_resonance.m',
 'line': 'function fit = fit_resonance(expData, binSize, nRes, header, kwargs)\n',
 'indent': '',
 'kwargs': {'header': "'none'",
  'type': '2',
  'globalFraction': '0.5',
  'forceGuess': 'false',
  'checkPlot': 'false',
  'gaussianFit': 'false',
  'gaussianFilter': '0',
  'smoothDegree': '2',
  'diamond': "'doublet'",
  'slopeCorrection': 'false',
  'crop': "'none'",
  'fcrop': 'false'}}

In [19]:
admin_tools.arguments.first_line_comments(save=True, debug=False)

C:\Users\micha\github\QDMlab\fitting\B111.m
----------------------------------------------------------------------------------------------------
NO CHANGE
----------------------------------------------------------------------------------------------------

C:\Users\micha\github\QDMlab\Bz_and_UpCont\B111ToBz_CommLine.m
----------------------------------------------------------------------------------------------------
NO CHANGE
----------------------------------------------------------------------------------------------------

C:\Users\micha\github\QDMlab\utilities\BinImage.m
----------------------------------------------------------------------------------------------------
NO CHANGE
----------------------------------------------------------------------------------------------------

C:\Users\micha\github\QDMlab\map_editing\FitMoment.m
----------------------------------------------------------------------------------------------------
NO CHANGE
----------------------------------------